In [ ]:
import time

from langchain_google_vertexai import ChatVertexAI

from resume_assistant.application.dataset.upload import upload_to_gcs
from resume_assistant.application.rag.retriever import get_langchain_docs
from resume_assistant.application.rag.utils import extract_content_from_documents
from resume_assistant.application.dataset.extraction import extract_job_description, scrape_webpage_content
from resume_assistant.models.model import get_model
import resume_assistant.application.rag.templates as templates 

In [ ]:
content = None
user_resume_path = "Olawale_Machine_Learning_Engineer_Copy.pdf"
url = "https://job-boards.greenhouse.io/scaleai/jobs/4413274005"

In [ ]:
public_url = upload_to_gcs(user_resume_path, content)
print("222")
documents = get_langchain_docs(user_resume_path)
content = extract_content_from_documents(documents)

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_google_community import DocAIParser
from langchain_core.document_loaders.blob_loaders import Blob


GCS_BUCKET_NAME = os.getenv("GCS_BUCKET_NAME")
DOC_PROCESSOR_NAME = os.getenv("DOC_PROCESSOR_NAME")

parser = DocAIParser(location="us",
                    processor_name=DOC_PROCESSOR_NAME,
                    gcs_output_path="gs://{}/resume-assistant/data/output/dev/pdfs".format(GCS_BUCKET_NAME))
inp_path = f"gs://{GCS_BUCKET_NAME}/resume-assistant/data/input/dev/pdfs/{user_resume_path}"
blob = Blob(path=inp_path)
operations = parser.docai_parse([blob])

In [ ]:
while parser.is_running(operations):
    time.sleep(0.5)
results = parser.get_results(operations)
docs = list(parser.parse_from_results(results))

In [ ]:
webpage_content = scrape_webpage_content(url)
job_description = extract_job_description(webpage_content)

In [ ]:
chat_model = get_model()
ats_chain = templates.ATS_TEMPLATE | chat_model 
ats_result = ats_chain.invoke({"job_description": job_description,
                       "resume_content": content})

In [ ]:
import resume_assistant.application.processing.postprocess as postprocess
json_ats_result = postprocess.get_json_from_result(ats_result)

In [ ]:
json_ats_result

In [ ]:
print(json_ats_result["Experience"])